In [17]:
import re
import numpy as np
import pandas as pd
import random 
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import warnings
# ignore some deprecation warnings
warnings.filterwarnings('ignore')

# Load main data set

In [2]:
df = pd.read_csv('retrosynthesis-all', header=None)
df['source'] = df[0].apply(lambda x: x.split('>>')[0])
df['target'] = df[0].apply(lambda x: x.split('>>')[1])
df.drop(0, axis=1, inplace=True)
df.head(3)

,source,target
0,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc...
1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1
2,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)...


# SMILES Vocabulary: How is it generated?

The model's Vocabulary handles the transformation of SMILES strings into a sequence of tokens. Tokens are the pre-defined lowest and indivisible unit of string text. In Natural Language Processing (NLP), tokens are typically defined on the word or character level. The level of tokenization dictates *what* the model can output, e.g., if tokenization on the character level is used, then the model outputs individual characters.

For generative SMILES models, tokenization is performed on the character level where each token *loosely* maps to a unique atom type (brackets, "(" for example indicate branching and thus, do not map to an atom but rather gives connectivity information).


In [4]:
import sys
sys.path.append('src/')
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, Vocabulary, create_vocabulary

tokenizer = SMILESTokenizer()
vocabulary = Vocabulary()

# create a vocabulary using all SMILES in df
smiles_dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
smiles_dataset = np.unique(smiles_dataset).tolist()

vocabulary = create_vocabulary(smiles_list=smiles_dataset, tokenizer=tokenizer)
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')

# construct an "Embedding layer"
EMBEDDING_DIM = 5
VOCAB_SIZE = len(vocabulary)
HIDDEN_SIZE = 3

There are 86 unique tokens in the vocabulary.



In [19]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        return hidden, cell


class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = VOCAB_SIZE

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

### We're ready to define everything we need for training our Seq2Seq model ###

# Training hyperparameters
num_epochs = 3
learning_rate = 0.001
batch_size = 2048

# Model hyperparameters
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size_encoder = VOCAB_SIZE
input_size_decoder = VOCAB_SIZE
output_size = VOCAB_SIZE
encoder_embedding_size = 5
decoder_embedding_size = 5
hidden_size = 6  # Needs to be the same for both RNN's
num_layers = 1
enc_dropout = 0.5
dec_dropout = 0.5


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sequence(tokenizer_array, desired_length=200):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

df['source'] = df['source'].apply(lambda x: tokenizer.tokenize(x, with_begin_and_end=True))
df['target'] = df['target'].apply(lambda x: tokenizer.tokenize(x, with_begin_and_end=True))

df['source'] = df['source'].apply(lambda x: vocabulary.encode(x))
df['target'] = df['target'].apply(lambda x: vocabulary.encode(x))

df['source'] = df['source'].apply(lambda x: pad_sequence(x))
df['target'] = df['target'].apply(lambda x: pad_sequence(x))

In [20]:
import torch
from torch.utils.data import DataLoader, Dataset

class TokenizedDataset(Dataset):
    def __init__(self, tokenized_df):
        self.source = tokenized_df['source']
        self.target = tokenized_df['target']

    def __len__(self):
        return len(self.source)

    def __getitem__(self, idx):
        return self.source[idx], self.target[idx]

# Assuming you have a tokenized dataframe called 'tokenized_df'
train_dataset = TokenizedDataset(df)

# Create the train iterator
train_iterator = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(f"Devide: {device}")
encoder_net = Encoder(
    input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout
).to(device)

decoder_net = Decoder(
    input_size_decoder,
    decoder_embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dec_dropout,
).to(device)

model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [116]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_idx, (input_batch, target_batch) in enumerate(tqdm(train_iterator)):
        # Transfer data to the appropriate device
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)

        # Forward pass
        optimizer.zero_grad()
        output_batch = model(input_batch, target_batch)

        # Reshape outputs and targets to match the loss function requirements
        output_batch = output_batch.view(-1, output_size)
        target_batch = target_batch.view(-1).long()  # Convert target tensor to torch.long

        # Calculate the loss
        loss = criterion(output_batch, target_batch)
        epoch_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()


    # Calculate the average loss for the epoch
    average_loss = epoch_loss / len(train_iterator)

    # Print the average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")


100%|██████████| 22/22 [00:26<00:00,  1.21s/it]


Epoch 1/3, Loss: 4.0365


100%|██████████| 22/22 [00:26<00:00,  1.21s/it]


Epoch 2/3, Loss: 3.8081


100%|██████████| 22/22 [00:26<00:00,  1.21s/it]

Epoch 3/3, Loss: 3.4770
